In [ ]:
import torch
from torch import nn

In [ ]:
def masked_softmax(X, valid_lens):  #@save
    """Perform softmax operation by masking elements on the last axis."""
    # X: 3D tensor, valid_lens: 1D or 2D tensor
    def _sequence_mask(X, valid_len, value=0):
        maxlen = X.size(1)
        mask = torch.arange((maxlen), dtype=torch.float32,
                            device=X.device)[None, :] < valid_len[:, None]
        X[~mask] = value
        return X

    if valid_lens is None:
        return nn.functional.softmax(X, dim=-1)
    else:
        shape = X.shape
        if valid_lens.dim() == 1:
            valid_lens = torch.repeat_interleave(valid_lens, shape[1])
        else:
            valid_lens = valid_lens.reshape(-1)
        # On the last axis, replace masked elements with a very large negative
        # value, whose exponentiation outputs 0
        X = _sequence_mask(X.reshape(-1, shape[-1]), valid_lens, value=-1e6)
        return nn.functional.softmax(X.reshape(shape), dim=-1)


In [36]:
test = masked_softmax(torch.rand(2, 2, 4), torch.tensor([[1, 3], [2, 4]]))
test

tensor([[[1.0000, 0.0000, 0.0000, 0.0000],
         [0.2091, 0.2600, 0.5309, 0.0000]],

        [[0.3023, 0.6977, 0.0000, 0.0000],
         [0.1612, 0.2475, 0.2262, 0.3650]]])